In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors
import csv
import json
import os

**读入预处理好的数据**

In [2]:
# 从文件load到numpy array
# 读入service_type==1的训练数据
x_train_1 = np.load('preprocessed/x_train_1.npy')
x_validation_1 = np.load('preprocessed/x_validation_1.npy')
y_train_1 = np.load('preprocessed/y_train_1.npy')
y_validation_1 = np.load('preprocessed/y_validation_1.npy')
# 读入service_type==4的训练数据
x_train_4 = np.load('preprocessed/x_train_4.npy')
x_validation_4 = np.load('preprocessed/x_validation_4.npy')
y_train_4 = np.load('preprocessed/y_train_4.npy')
y_validation_4 = np.load('preprocessed/y_validation_4.npy')
# 读入测试数据
x_test = np.load('preprocessed/x_test.npy')
id_test = np.load('preprocessed/id_test.npy')

print(x_test[0])
print(id_test[0])


[1.00000000e+00 0.00000000e+00 1.80000000e+01 5.04000000e+01
 1.08200000e+02 1.18900000e+02 1.77700000e+02 3.00000000e+02
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.00000000e+00 2.00000000e+00 6.00000000e+01 0.00000000e+00
 2.63908486e+03 3.77833333e+01 3.77833333e+01 0.00000000e+00
 1.00000000e+00 1.90000000e+01 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
012pSke7DsCrR985


**模型训练**

In [3]:
# 然后导入决策树类，并为其初始化2个实例。
from sklearn import tree

# 创建service_type==1的估计器
estimator_1 = tree.DecisionTreeClassifier()
print('estimator_1 finished')
# 创建好之后，开始用训练数据进行训练。
estimator_1.fit(x_train_1, y_train_1)
print('fit_1 finished')
# 在验证集service_type==1上的准确率
score_1 = estimator_1.score(x_validation_1,y_validation_1)
print('service_type==1 validation acc: '+str(score_1))

# 创建service_type==4的估计器
estimator_4 = tree.DecisionTreeClassifier()
print('estimator_4 finished')
# 创建好之后，开始用训练数据进行训练。
estimator_4.fit(x_train_4, y_train_4)
print('fit_4 finished')
# 在验证集service_type==4上的准确率
score_4 = estimator_4.score(x_validation_4,y_validation_4)
print('service_type==4 validation acc: '+str(score_4))

estimator_1 finished
fit_1 finished
service_type==1 validation acc: 0.9902552649486155
estimator_4 finished
fit_4 finished
service_type==4 validation acc: 0.824547329818932


**模型验证**

In [4]:
#在整个验集上的准确率
acc = score_1*len(x_validation_1)
acc += score_4*len(x_validation_4)
acc /= (len(x_validation_1)+len(x_validation_4))
print('whole validation acc: '+str(acc))

whole validation acc: 0.9019127063002004


**模型测试**

In [5]:
# 接着用测试集测试算法。
y_predicted = []
for i,query in enumerate(x_test):
    if query[0] == 4:
        y_predicted.append(np.int64(estimator_4.predict([query[1:]])))
    else:
        # 测试集中的service_type除了1或4还存在3
        y_predicted.append(np.int64(estimator_1.predict([query[1:]])))

print('predict finished')
y_predicted[0]

predict finished


90063345

In [6]:
header = [('user_id','current_service')]

with open('deci_submission.csv', 'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    writer.writerows(header)
    for i in range(len(x_test)):
        writer.writerows([(id_test[i],y_predicted[i])])
    f.close()

In [7]:
# 保存成python支持的文件格式pickle
import pickle
with open('deci_1.pickle', 'wb') as fw:
    pickle.dump(estimator_1, fw)
with open('deci_4.pickle', 'wb') as fw:
    pickle.dump(estimator_4, fw)
    
## 如需加载.pickle
#with open('deci.pickle', 'rb') as fr:
#    new_estimator = pickle.load(fr)
#    print new_estimator.predict(x_test)